# Import packages

In [ ]:
#-*- coding: utf-8 -*-

import numpy as np
import pandas as pd
from sklearn import linear_model # packages for the logistic regression function to plot the logistic regression 
from sklearn.linear_model import LogisticRegression # packages for the logistic regression function to plot the logistic regression 
from scipy import stats
from scipy.stats.stats import pearsonr # Pearson's correlation
from pandas.core.frame import DataFrame as DF
from copy import copy as copy
import operator as operator
import pylab
import scipy.io as spi

# Show graphs in Notebook
%matplotlib inline

# linking to files in notebooks
from IPython.display import FileLink, FileLinks

# Plotting tools
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import seaborn as sns
%pylab inline
figsize(15, 15);

# Set up interface with R
import rpy2
%reload_ext rpy2.ipython

# GLM in python
import patsy
import statsmodels.api as sm
import statsmodels.formula.api as smf
import statsmodels.stats as sm_stats

# Make it easy to set and find values in a multi-index DF
idx = pd.IndexSlice

# Useful to quickly extract csv files
import glob

Populating the interactive namespace from numpy and matplotlib


`%matplotlib` prevents importing * from pylab and numpy


RuntimeError: Unable to locate R.dll within C:\Program Files\R\R-3.3.2

# Homemade functions

In [ ]:
def ExtractConf(FullData, Bin, Correct, Response):
    Conf = FullData.loc[(FullData['CorrectKey']== Correct) & (FullData['Conf_Bin']==Bin)].groupby('participant')['Response'].value_counts()
    return Conf.loc[slice(None), Response]

# This is the final function thet is using the other functions to generate the binned data, it requires three parameters,
# group (monolingual or bilingual, or more generally S1 or S2) correct (left or right, or S1 or S2), and number of bins
# for the confidence.
def GenConfBins(FullData, Correct):
    Bins=int(FullData['Conf_Bin'].max())
#     print (Bins)
    # First we create an output array with twice the amount of  columns that there are bins (first for when the
    # participants answer S1, and then for when they answer S2) and 31 rows (one per participant in each group)
    OutputArray = [ [1]*(Bins*2) for i in FullData['participant'].unique()]
    # The data is generated in order, from left to right, both in terms of responses and the positions in the output
    
    #Starting with the left, S1, responses, we loop through the bins:
    for Bin in range(Bins):
        # We generate a pandas series for the binned data in this particular confidence/response bin for each participant
        # The (Bins-Bin) expression means that we start with the highest value bin and work our way down
        inputseries = ExtractConf(FullData, (Bins - Bin), Correct, 'left')
        # We generate a list of the participants who have a non-zero value in this bin
        participants = inputseries.index.get_values()
#         print ( participants)
        # A loop that maps the binned value of a given participant to the correct place in the output array
        for participant in participants:
            OutputArray[participant-1][Bin] = 1 + inputseries.loc[participant]
            
    #Next we do the right, S2, responses, we loop through the bins:
    for Bin in range(Bins):
        # We generate a pandas series for the binned data in this particular confidence/response bin for each participant
        # On this side we start with the lowest confidence bin and work our way up
        inputseries = ExtractConf(FullData, Bin+1, Correct, 'right')
        # We generate a list of the participants who have a non-zero value in this bin
        participants = inputseries.index.get_values()
        # A loop that maps the binned value of a given participant to the correct place in the output array
        # The (Bins+Bin) expression is because we have already filled the first set of array with the left (S1) values
        for participant in participants:
            OutputArray[participant-1][(Bins+Bin)] = 1 + inputseries.loc[participant]
                     
    return OutputArray

In [ ]:
def zscore(data, old_var):
    output = ((data[old_var] - data[old_var].mean())/
              data[old_var].std())
    return output.values

In [ ]:
def participant_zscore(data, old_var, sort_var):
    data2 = data.set_index(sort_var).copy()
    output = ((data2[old_var] - data2.groupby(level=sort_var)[old_var].mean())/
              data2.groupby(level=sort_var)[old_var].std())
    return output.values

# Making the dataframe

<font size=4>[Skip this section and load pre-cleaned and combined data](#load_data)</font>

## Loading the data

In [ ]:
data_path = "C:\\Users\\user\\Desktop\\실험 데이터\\Data\\Main"
file_list = glob.glob("C:\\Users\\user\\Desktop\\실험 데이터\\Data\\Main\\With name\\*.csv")
#print (file_list)
data_frames = [pd.read_csv(file, encoding = "ISO-8859-1") for file in file_list]
# print (data_frames)

## Cleaning dataframe

In [ ]:
clean = []
for data_frame in data_frames:
    columns = data_frame.columns[13:16] | data_frames[0].columns[21:23] | data_frame.columns[32:33] | data_frames[0].columns[23:25]
    print (columns)
#     ORIGINAL - columns = data_frame.columns[13:21] | data_frames[0].columns[23:25] | data_frame.columns[27:32] | data_frame.columns[32:33] 
#     data_frame.columns[35:36]
    main_trials_start = data_frame.loc[data_frame['Instructions4Response.keys'] == 'space'].index[0]+1
    clean.append(data_frame.loc[main_trials_start:, columns].copy())
#     print(clean)
#    print ('participant', data_frame['participant'].mean())
#     print (columns)
#     print (data_frame)

In [ ]:
data = pd.concat(clean)
print (data.columns)
print (data['participant'].unique())

## Preparing the data for matlab

### Binning the data at the participant level

In [ ]:
data = data.sort_values(by=['participant'],ascending=[True])
print (data['participant'].unique())
data['Conf_Bin'] = np.nan
# print (data.columns)
# save concatinated data
# df = pd.DataFrame(data, columns = data.columns)
# print (df)
# df.to_csv('example.csv')

for participant in data.participant.unique():
    data.loc[data['participant']==participant, 'Conf_Bin'] = pd.cut(data.loc[data['participant']==participant, 'Confidence'], 3, labels=range(1, 4)).values
#     print (participant)

In [ ]:
data['Conf_Bin'].value_counts()
print (data['participant'].unique())
print (data['Conf_Bin'].value_counts())

### Preparing the dataframes

In [ ]:
# participant_nr = 1
# for participant in data.participant.unique():
#     print (participant)
#     data.loc[data['participant']==participant, 'participant'] = participant_nr
#     participant_nr += 1
# data['participant'].unique()
print (data['participant'].unique())
print (data)

In [ ]:
data_S1 = GenConfBins(data, 'left')
# print (data_S1)

In [ ]:
data_S2 = GenConfBins(data, 'right')
data_S2

In [ ]:
data_output = dict(zip(['S1', 'S2'], [data_S1, data_S2]))

### Checking that the output looks sensible

In [ ]:
data_output

### Saving the output

In [ ]:
spi.savemat('C:\\Users\\user\\Desktop\\metaExpdata.mat', data_output)
# open from matlab

## Loading the data from Matlab

In [ ]:
# # Loading the matlab files
# meta_conf = spi.loadmat("C:\\Users\\user\\Desktop\\metaExpdata.mat")

# # Extracting variables
# meta_da = meta_conf['fit']['meta_da'].tolist()[0][0][0]
# da = meta_conf['fit']['da'].tolist()[0][0][0]
# mratio = meta_conf['fit']['Mratio'].tolist()[0][0][0]

# #Generating the DF
# meta_data = DF([da, meta_da, mratio])
# meta_data = meta_data.transpose()
# meta_data.columns = ['Da', 'Meta_Da', 'Mratio']
# meta_data['participant'] = np.append(range(1, 31), range(71, 73))
# meta_data['Mrato_Log'] = np.log(meta_data['Mratio'])
# meta_data['Group'] = 'Monolingual'


In [ ]:
# meta_data = pd.concat([meta_data], ignore_index=True)